In [1]:
import numpy as np
from models.tennis_lstm import TennisLSTM
from models.loss_functions import weighted_loss, auto_weighted_loss
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataloaders.vanilla_slam_loaders import YearOpenSplitLoader, YearOpenSplitDataSet
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
import json
from dataloaders.valid_data_fields import valid_fields
from metrics.base_metrics import get_interval_success_rates

In [2]:

test_slam_years=['2014-ausopen',
                 '2014-frenchopen', '2014-usopen', '2014-wimbledon',
                ]
test_data_set = YearOpenSplitDataSet(test_slam_years)
test_data_loader = DataLoader(test_data_set, batch_size=1, shuffle=False, num_workers=4)

['2014-ausopen', '2014-frenchopen', '2014-usopen', '2014-wimbledon']


/Users/sven/miniconda3/envs/deep-tennis/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


dropped 0 matches
dropped 32 matches
dropped 6 matches
dropped 2 matches


In [3]:
from test_model import test_my_model
test_model_info = {'input_dim':len(valid_fields), 'hidden_dim':25, 
          'batch_size': 1, 
          'predict_mask':True, 
          'num_layers':1}

test_model = TennisLSTM(**test_model_info)
state_dict = torch.load('saved_models/tennis_lstm-1575242236.280942.pt')
print(state_dict.keys())
test_model.load_state_dict(state_dict)
print(print(test_model.__dict__))

print("testing within loop")
interval_metrics = get_interval_success_rates(test_model, test_data_loader)


odict_keys(['lstm.weight_ih_l0', 'lstm.weight_hh_l0', 'lstm.bias_ih_l0', 'lstm.bias_hh_l0', 'linear.weight', 'linear.bias'])
{'training': True, '_parameters': OrderedDict(), '_buffers': OrderedDict(), '_backward_hooks': OrderedDict(), '_forward_hooks': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_state_dict_hooks': OrderedDict(), '_load_state_dict_pre_hooks': OrderedDict(), '_modules': OrderedDict([('lstm', LSTM(47, 25)), ('linear', Linear(in_features=25, out_features=2, bias=True))]), 'input_dim': 47, 'hidden_dim': 25, 'batch_size': 1, 'num_layers': 1, 'predict_mask': True, 'prematch_probs': None}
None
testing within loop
testing on 812 examples
at 0.5 way through the match, predicted 585 out of 812 correctly
at 0.75 way through the match, predicted 602 out of 812 correctly
at 1 way through the match, predicted 611 out of 812 correctly
predicted 0.6527873576955384 of all points correctly


In [4]:
for i, data in enumerate(test_data_loader):
    X_train, prematch_probs, y_train  = data 
    X_train = X_train.float()
    y_train = y_train.float()
    prematch_probs = prematch_probs.float()
    # print(model.hidden)
    # model.hidden = model.init_hidden(prematch_probs)
    test_model.set_prematch_probs(prematch_probs)

    y_pred, mask = test_model(X_train)
    print(X_train, y_pred, y_train)
    if i > 10:
        break


tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [1., 0., 0.,  ..., 1., 1., 1.],
         [1., 0., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 5., 4.,  ..., 1., 1., 1.],
         [1., 5., 4.,  ..., 1., 1., 1.],
         [1., 6., 4.,  ..., 1., 0., 1.]]]) tensor([0.5365, 0.5343, 0.5263, 0.5322, 0.5366, 0.5882, 0.5592, 0.5464, 0.4978,
        0.4835, 0.5628, 0.5203, 0.5200, 0.5231, 0.6039, 0.5482, 0.4710, 0.3927,
        0.5566, 0.5284, 0.5280, 0.5275, 0.5322, 0.5400, 0.5520, 0.4953, 0.4925,
        0.4782, 0.4695, 0.4812, 0.5276, 0.5274, 0.5403, 0.5340, 0.5474, 0.5001,
        0.5370, 0.4503, 0.4803, 0.4772, 0.4840, 0.4789, 0.5184, 0.4156, 0.4693,
        0.4721, 0.4769, 0.4756, 0.4907, 0.4990, 0.4655, 0.4665, 0.4567, 0.4594],
       grad_fn=<ViewBackward>) tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.,

tensor([[[ 1.,  0.,  0.,  ...,  3., 13., 13.],
         [ 1.,  0.,  0.,  ...,  3., 13., 13.],
         [ 1.,  0.,  0.,  ...,  3., 13., 13.],
         ...,
         [ 4.,  5.,  2.,  ...,  2.,  1.,  7.],
         [ 4.,  5.,  2.,  ...,  2.,  1.,  7.],
         [ 4.,  6.,  2.,  ...,  2.,  0.,  7.]]]) tensor([0.5071, 0.5143, 0.5188, 0.5706, 0.5330, 0.5387, 0.5438, 0.6135, 0.5722,
        0.5410, 0.5564, 0.5619, 0.5627, 0.6377, 0.5841, 0.5360, 0.5417, 0.5452,
        0.6372, 0.6189, 0.6111, 0.6108, 0.6256, 0.5681, 0.5082, 0.4990, 0.4914,
        0.4958, 0.4858, 0.4839, 0.6448, 0.6290, 0.4888, 0.4976, 0.5372, 0.6428,
        0.6377, 0.6100, 0.4982, 0.4894, 0.6502, 0.6173, 0.4882, 0.4728, 0.4856,
        0.4793, 0.4877, 0.5054, 0.4846, 0.4782, 0.6133, 0.5229, 0.4759, 0.4846,
        0.4745, 0.4838, 0.6247, 0.5861, 0.5202, 0.4959, 0.5723, 0.5106, 0.4708,
        0.4682, 0.5045, 0.5828, 0.5855, 0.5712, 0.5705, 0.5530, 0.5424, 0.5317,
        0.5300, 0.5252, 0.5270, 0.4794, 0.4628, 0.4616, 0.4606

In [5]:
print(test_model.lstm.__dict__)

{'training': True, '_parameters': OrderedDict([('weight_ih_l0', Parameter containing:
tensor([[ 0.1030, -0.0883, -0.0388,  ..., -0.1054,  0.1028, -0.1062],
        [ 0.0588, -0.0577, -0.0219,  ..., -0.1500,  0.0402,  0.0480],
        [-0.1341, -0.0959,  0.0683,  ..., -0.1370, -0.1682, -0.0919],
        ...,
        [ 0.0684,  0.0271,  0.0702,  ...,  0.0606,  0.1089,  0.0369],
        [ 0.0857,  0.1575,  0.1065,  ...,  0.0293, -0.0377,  0.1529],
        [ 0.0140, -0.1595, -0.0326,  ...,  0.0900,  0.1040,  0.1219]],
       requires_grad=True)), ('weight_hh_l0', Parameter containing:
tensor([[ 0.0127,  0.0107,  0.1160,  ...,  0.1752, -0.0293,  0.0269],
        [-0.1958,  0.1667,  0.0157,  ...,  0.1008, -0.0762,  0.1442],
        [ 0.1925,  0.1691, -0.1014,  ...,  0.1419,  0.1277, -0.0653],
        ...,
        [ 0.0388, -0.1230, -0.0966,  ..., -0.1187, -0.1099,  0.1352],
        [ 0.1598,  0.1524,  0.1759,  ..., -0.0792, -0.0985,  0.0933],
        [ 0.0938,  0.1252,  0.0584,  ...,  0.0548

In [6]:
list(test_model.lstm.parameters())

[Parameter containing:
 tensor([[ 0.1030, -0.0883, -0.0388,  ..., -0.1054,  0.1028, -0.1062],
         [ 0.0588, -0.0577, -0.0219,  ..., -0.1500,  0.0402,  0.0480],
         [-0.1341, -0.0959,  0.0683,  ..., -0.1370, -0.1682, -0.0919],
         ...,
         [ 0.0684,  0.0271,  0.0702,  ...,  0.0606,  0.1089,  0.0369],
         [ 0.0857,  0.1575,  0.1065,  ...,  0.0293, -0.0377,  0.1529],
         [ 0.0140, -0.1595, -0.0326,  ...,  0.0900,  0.1040,  0.1219]],
        requires_grad=True), Parameter containing:
 tensor([[ 0.0127,  0.0107,  0.1160,  ...,  0.1752, -0.0293,  0.0269],
         [-0.1958,  0.1667,  0.0157,  ...,  0.1008, -0.0762,  0.1442],
         [ 0.1925,  0.1691, -0.1014,  ...,  0.1419,  0.1277, -0.0653],
         ...,
         [ 0.0388, -0.1230, -0.0966,  ..., -0.1187, -0.1099,  0.1352],
         [ 0.1598,  0.1524,  0.1759,  ..., -0.0792, -0.0985,  0.0933],
         [ 0.0938,  0.1252,  0.0584,  ...,  0.0548, -0.1901, -0.0948]],
        requires_grad=True), Parameter conta